In [2]:

from train_cnn import TrainingModel, init_training_params, train_cnn, TRANSFORM, ModelWithTransform, get_image_data, visualize_loss_curve, visualize_first_layer_weights
import torch
n_samples = 1000
output_file_path = "test_model.pth"

print(f"Using {n_samples} images for training and testing.")

model = TrainingModel()
training_params = init_training_params(model, device = torch.device("cuda"))

# resize to 224x224 as preprocessing step
model_and_transform = ModelWithTransform(model, TRANSFORM)

train,val,test = get_image_data(n_samples)

print(len(train))


for i in range(0, len(train)):
    if train[i][1]:
        random_fire_img = TRANSFORM(train[i][0])
        break



train_loss, val_loss ,fire_detector = train_cnn(model_and_transform,training_params,train, val)
# Save the trained model to a file
fire_detector.save_to_file(output_file_path)
print(f"Model saved to {output_file_path}")





Using 1000 images for training and testing.
0
Begin training...


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
visualize_loss_curve(train_loss, val_loss)



In [ ]:
visualize_first_layer_weights(fire_detector.model.trained_model, random_fire_img)
